<a href="https://colab.research.google.com/github/DavidFranciscoDSA/ImersaoIA_Alura_Google/blob/main/PROJETOImersaoIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AULA 5 - Imersão IA Alura + Google

In [27]:
!pip install -q -U google-generativeai
!pip install -q -U pypdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
apiK = userdata.get('SECRET_KEY')

# Criando o Gemini

In [3]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key = apiK )

# Modelos Disponiveis

In [7]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


# Importação bibliotecas

In [5]:
import pandas as pd
import numpy as np

# Exemplos de embedding

In [10]:
texto = input('Digite para gerar o embedding: ')
resultado = genai.embed_content(model="models/embedding-001", content=texto)
print(resultado['embedding'])
print(len(resultado['embedding']))

Digite para gerar o embedding: preciso saber quantos dias 
[0.005255208, -0.004374492, -0.021008262, 0.0080795055, 0.03681873, 0.022226483, 0.03395751, -0.016656635, 0.051615182, 0.030711018, -0.009872081, 0.034493085, 0.014394191, -0.029162701, -0.041317638, -0.013029748, -0.0036037688, 0.0064135157, 0.0055608763, -0.0042380486, -0.008730696, 0.052502174, -0.017172266, 0.020157378, 0.008238256, -0.043672595, -0.004868965, -0.024920408, 0.0016458244, 0.056295242, -0.07359954, 0.08104163, -0.022270659, 0.020815885, 0.02077223, -0.058320865, 0.0018146011, 0.01060399, -0.004855481, 0.050669506, 0.0141344145, -0.0354933, 0.0001985003, -0.025976975, 0.02571827, 0.036619425, 0.0026654436, 0.0020924858, 0.010451327, -0.035407, 0.025563266, -0.03617905, 0.061977286, -0.029456923, -0.0003023013, -0.061494198, 0.045547277, -0.018597705, -0.01571514, 0.02055666, 0.021485148, 0.03427374, -0.02650083, 0.082434915, -0.023126898, -0.022775223, -0.013188874, -0.00892964, 0.07117003, -0.03702288, 0.020

Embedding de documentos que serão buscados

# Coletando arquivos pdf e particionando caso o valor exceda 10000 bytes

In [51]:
import PyPDF2

def quebrar_pdf(nomeParticao, nome_arquivo_pdf):
    # Nome do arquivo PDF base
    nome_arquivo_base = nomeParticao

    # Tamanho máximo do arquivo em bytes
    tamanho_maximo = 10000

    # Inicializar variáveis
    texto_atual = ""
    numero_arquivo = 1
    pdf_file = open(f'{nome_arquivo_base}_{numero_arquivo}.pdf', 'wb')
    pdf_writer = PyPDF2.PdfWriter()

    # Abrir o arquivo PDF original para leitura
    pdf_file_original = open(nome_arquivo_pdf, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file_original)

    # Iterar pelas páginas do PDF original
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        texto_pagina = page.extract_text()

        # Verificar se o tamanho máximo do arquivo seria excedido
        if len(texto_atual.encode('utf-8')) + len(texto_pagina.encode('utf-8')) > tamanho_maximo:
            # Fechar o arquivo atual
            pdf_file.close()

            # Criar um novo arquivo
            numero_arquivo += 1
            pdf_file = open(f'{nome_arquivo_base}_{numero_arquivo}.pdf', 'wb')
            pdf_writer = PyPDF2.PdfWriter()

            # Reiniciar o texto atual
            texto_atual = ""

        # Adicionar texto ao arquivo atual
        texto_atual += texto_pagina
        pdf_writer.add_page(page)
        pdf_writer.write(pdf_file)

    # Fechar o arquivo final
    pdf_file.close()
    pdf_file_original.close()

    print(f'Texto dividido em {numero_arquivo} arquivos.')

In [52]:
nomeParticao = 'codConsumidor'
nome_arquivo_pdf = 'cdc_e_normas_correlatas_2ed.pdf'
quebrar_pdf(nomeParticao, nome_arquivo_pdf)

Texto dividido em 53 arquivos.


# Transformando o pdf em texto

In [90]:
def converterPDF_texto(arquivoPDF,numero_arquivo):
   lista_texto = []
   for i in range(1,numero_arquivo+1):
      # Abrir o arquivo PDF
      arquivoPDF = f"{arquivoPDF}{i}.pdf"
      print(arquivoPDF)
      pdf_file = open(arquivoPDF, 'rb')
      pdf_reader = PyPDF2.PdfReader(pdf_file)

      # Extrair texto de todas as páginas
      texto_completo = ""
      for page_num in range(len(pdf_reader.pages)):
          page = pdf_reader.pages[page_num]
          texto_completo += page.extract_text()
          texto_completo = texto_completo.replace('\n', ' ')
          lista_texto.append(texto_completo)

      # Fechar o arquivo
      pdf_file.close()

   # Imprimir o texto extraído
   return lista_texto

In [91]:
arquivoPDF = 'codConsumidor_'
numero_arquivo = 53
lista = converterPDF_texto(arquivoPDF,numero_arquivo)

print(len(lista))

codConsumidor_1.pdf
codConsumidor_1.pdf2.pdf


FileNotFoundError: [Errno 2] No such file or directory: 'codConsumidor_1.pdf2.pdf'

In [43]:
#Listagem de documentos que serão buscados

DOCUMENTO1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema, use os controles no painel ou os comandos de voz."
}

DOCUMENTO2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, música e configurações do carro."
}

DOCUMENTO3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada."
}

DOCUMENTO4 = {
    "Título": "Código de defesa do consumidor",
    "Conteúdo": codigoDefesa
}

documents = [DOCUMENTO1, DOCUMENTO2, DOCUMENTO3,DOCUMENTO4 ]

# Modificando para um dataframe

In [44]:
df = pd.DataFrame(documents)
df.columns = ['Titulo', 'Conteudo']
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...
3,Código de defesa do consumidor,2a edição Atualizada até setembro de 2017Códig...


# Usando em funções

In [45]:
model = "models/embedding-001"
def embed_fn(title, text):
  return genai.embed_content(model="models/embedding-001",
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")['embedding']


# Adicionando a coluna embedding no dataframe acima

In [47]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row['Titulo'], row['Conteudo']),axis=1)

BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/embedding-001:embedContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 10000 bytes.

# Pergunta com resposta gerada a partir do contexto do embedding

In [48]:
def gerar_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")['embedding']
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [49]:
consulta = 'direito do consumidor'
resposta = gerar_buscar_consulta(consulta, df, model)
print(resposta)

KeyError: 'Embeddings'